In [ ]:
"""
Originally from train_stan4.ipynb

Summary: 

"""

'\nOriginally from train_stan4.ipynb\n\nSummary: \n\n'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
% cd /content/gdrive/My Drive/Github/"CS 499 - SPRING 2022"/"3. Experiments"/"2. STAN-Without Missing Data"
! pip install epiweeks
! pip install haversine
! pip install dgl

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Github/CS 499 - SPRING 2022/3. Experiments/2. STAN-Without Missing Data


In [ ]:
"""
Import libraries needed
"""
import torch
import numpy as np
import matplotlib.pyplot as plt

from preprocess_data_library2 import get_preprocessed_data
from model5 import STAN_v5
from torch import nn

In [ ]:
import pickle

# # Get preprocessed data and unpackage variables needed during training/testing/validation
# preprocessed_data = get_preprocessed_data()
# # Save preprocessed data as pickle so we don't have to consecutively re-download the same data (time consuming) when re-running all cells in this notebook
# with open('preprocessed_data.pickle', 'wb') as handle:
#     pickle.dump(preprocessed_data, handle)

# Load in saved preprocessed_data
preprocessed_data = None
with open('preprocessed_data.pickle', 'rb') as handle:
    preprocessed_data = pickle.load(handle)

In [ ]:
training_variables = preprocessed_data['training_variables']
validation_variables = preprocessed_data['validation_variables']
testing_variables = preprocessed_data['testing_variables']

static_feat = preprocessed_data['static_feat']
loc_list = preprocessed_data['loc_list']
g = preprocessed_data['graph']

train_x = training_variables['train_x']
train_y_confirmed = training_variables['train_y_confirmed']
train_y_deaths = training_variables['train_y_deaths']
train_y_confirmed_change = training_variables['train_y_confirmed_change']
train_y_deaths_change = training_variables['train_y_deaths_change']

val_x = validation_variables['val_x']
val_y_confirmed = validation_variables['val_y_confirmed']
val_y_deaths = validation_variables['val_y_deaths']
val_y_confirmed_change = validation_variables['val_y_confirmed_change']
val_y_deaths_change = validation_variables['val_y_deaths_change']

test_x = testing_variables['test_x']
test_y_confirmed = testing_variables['test_y_confirmed']
test_y_deaths = testing_variables['test_y_deaths']
test_y_confirmed_change = testing_variables['test_y_confirmed_change']
test_y_deaths_change = testing_variables['test_y_deaths_change']

In [ ]:
history_window = 6 # Redeclared from preprocess_data_library
pred_window = 15 # Redeclared from preprocess_data_library
slide_step = 5 # Redeclared from preprocess_data_library

#Build STAN model

in_dim = 4*history_window
hidden_dim1 = 32
hidden_dim2 = 32
gru_dim = 32
num_heads = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

g = g.to(device)
model = STAN_v5(g, in_dim, hidden_dim1, hidden_dim2, gru_dim, num_heads, pred_window, device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.MSELoss()

In [ ]:
model

STAN_v5(
  (layer1): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=24, out_features=32, bias=True)
        (attn_fc): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (layer2): MultiHeadGATLayer(
    (heads): ModuleList(
      (0): GATLayer(
        (fc): Linear(in_features=32, out_features=32, bias=True)
        (attn_fc): Linear(in_features=64, out_features=1, bias=True)
      )
    )
  )
  (gru): GRUCell(32, 32)
  (nn_cumulative_confirmed): Linear(in_features=32, out_features=15, bias=True)
)

In [ ]:
train_x = torch.tensor(train_x).to(device)
train_y_confirmed = torch.tensor(train_y_confirmed).to(device)
train_y_deaths = torch.tensor(train_y_deaths).to(device)
train_y_confirmed_change = torch.tensor(train_y_confirmed_change).to(device)
train_y_deaths_change = torch.tensor(train_y_deaths_change).to(device)

val_x = torch.tensor(val_x).to(device)
val_y_confirmed= torch.tensor(val_y_confirmed).to(device)
val_y_deaths = torch.tensor(val_y_deaths).to(device)
val_y_confirmed_change = torch.tensor(val_y_confirmed_change).to(device)
val_y_deaths_change = torch.tensor(val_y_deaths_change).to(device)

test_x = torch.tensor(test_x).to(device)
test_y_confirmed = torch.tensor(test_y_confirmed).to(device)
test_y_deaths = torch.tensor(test_y_deaths).to(device)
test_y_confirmed_change = torch.tensor(test_y_confirmed_change).to(device)
test_y_deaths_change = torch.tensor(test_y_deaths_change).to(device)

N = torch.tensor(static_feat[:, 0], dtype=torch.float32).to(device).unsqueeze(-1)

In [ ]:
#Train STAN_v5

all_loss = []
file_name = './save/stan'
# min_loss = 1e10
min_loss = 1e20

loc_name = 'Kentucky'
cur_loc = loc_list.index(loc_name)

for epoch in range(1):
    model.train()
    optimizer.zero_grad()
    
    # cumulative_confirmed, cumulative_deaths, _ = model(train_x)
    # cumulative_confirmed, _ = model(train_x)
    predicted_confirmed_change, _, debug_messages = model(train_x)
    print(debug_messages)

    # loss = criterion(cumulative_confirmed.squeeze(), train_y_confirmed[cur_loc])+criterion(cumulative_deaths.squeeze(), train_y_deaths[cur_loc])
    # loss = criterion(cumulative_confirmed.squeeze(), train_y_confirmed[cur_loc])
    loss = criterion(predicted_confirmed_change.squeeze(), train_y_confirmed_change[cur_loc])

    loss.backward()
    optimizer.step()
    all_loss.append(loss.item())
    
    model.eval()
    # _, _, prev_h = model(train_x)
    _, prev_h, _ = model(train_x)

    # val_cumulative_confirmed, val_cumulative_deaths, _ = model(val_x, prev_h)
    # val_cumulative_confirmed, _ = model(val_x, prev_h)
    val_predicted_confirmed_change, _, _ = model(val_x, prev_h)

    # val_loss = criterion(val_cumulative_confirmed.squeeze(), val_y_confirmed[cur_loc]) + criterion(val_cumulative_deaths, val_y_deaths[cur_loc])
    # val_loss = criterion(val_cumulative_confirmed.squeeze(), val_y_confirmed[cur_loc])
    val_loss = criterion(val_predicted_confirmed_change.squeeze(), val_y_confirmed_change[cur_loc])

    if val_loss < min_loss:    
        state = {
            'state': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(state, file_name)
        min_loss = val_loss
        print('-----Save best model-----')
    
    print('Epoch %d, Loss %.2f, Val loss %.2f'%(epoch, all_loss[-1], val_loss.item()))

ValueError: ignored

In [ ]:
#Pred with STAN
file_name = './save/stan'
checkpoint = torch.load(file_name)
model.load_state_dict(checkpoint['state'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

# Get hidden state from training/validation for use in testing
prev_x = torch.cat((train_x, val_x), dim=1)
# _, _, h = model(prev_x)
_, h = model(prev_x)


# Get predicted values for cumulative confirmed and cumulative deaths
# cumulative_confirmed, cumulative_deaths, _ = model(test_x, h)
# cumulative_confirmed, _ = model(test_x, h)
predicted_change_confirmed, _ = model(test_x, h)

# Real world reported/raw values for change in # confirmed is test_y_confirmed_change

# Prepare predicted and true values for plotting
# predicted_confirmed = cumulative_confirmed.squeeze().detach().numpy()
# predicted_deaths = cumulative_deaths.squeeze().detach().numpy()
predicted_output = predicted_change_confirmed.squeeze().detach().numpy()

# true_cumulative_confirmed = test_y_confirmed[cur_loc].squeeze()
# true_cumulative_deaths = test_y_deaths[cur_loc].squeeze()
true_confirmed_change = test_y_confirmed_change[cur_loc].squeeze()

In [ ]:
# plt.plot(true_cumulative_confirmed,c='r', label='Ground truth 1')
# plt.plot(predicted_confirmed,c='b', label='Prediction 1')
plt.plot(true_confirmed_change, c='r', label='Ground truth')
plt.plot(predicted_output, c='b', label='Prediction 1')

plt.legend()
plt.show()

print(true_confirmed_change)
print(predicted_output)

In [ ]:
print(predicted_confirmed)
# print(predicted_deaths)
print(true_cumulative_confirmed)
# print(true_cumulative_deaths)